In [36]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pickle
import string
import spacy
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import word_tokenize
np.random.seed(500)


In [37]:
scriptdata = pd.read_csv("C:/Users/Savita/Desktop/ml_prediction/prediction_gender/data/raw_data.csv")
scriptdata.head(2)

,_id,allbrowsercookies,bot,brand,browser,cookies.XSRF-TOKEN,cookies.__cfduid,cookies.cyber_astro_session,date,description,...,os,page_url,pc,referer,tablet,time,title,touch,user_id,visit_num
0,5ddb4e1a90281a177983e57c,"[""__gads=ID=4ac600ce86bc98a5:T=1569838984:S=AL...",False,NaN,Chrome,NaN,d74390cf7ecf49d02be7084c106884ffd1574653464,NaN,26-11-19,description,...,Windows,https://www.facultyplus.com/,True,https://www.facultyplus.com/,False,9:14:23,FacultyPlus - Indian Faculty Jobs - Assistant ...,False,MjY3_117.240.189.2,25.0
1,5ddb4e2b90281a177983e598,"[""__gads=ID=4ac600ce86bc98a5:T=1569838984:S=AL...",False,NaN,Chrome,NaN,d0ce46f9b728fd9e5ef5ec2b36a0bffac1574653481,NaN,26-11-19,description,...,Windows,https://www.facultyplus.com/engineering,True,https://www.facultyplus.com/engineering,False,9:14:39,engineering Archives - FacultyPlus,False,MjY3_117.240.189.2,26.0


In [38]:
scriptdata = scriptdata.drop(['_id','allbrowsercookies','bot','brand','browser','cookies.XSRF-TOKEN','cookies.__cfduid','cookies.cyber_astro_session','date','description','device','device_address','google_id','host_name',
 'image_text','impression','ip','language','local_hostname','location.IPv4','location.city','location.country_code','location.country_name','location.latitude','location.longitude','location.postal','location.state','mobile','model','mylocalstorage',
 'mysession','os','page_url','pc','referer','tablet','time','title','touch','user_id','visit_num'], axis = 1)

In [39]:
import string
scriptdata = scriptdata.apply(lambda x: x.astype(str).str.lower())


In [40]:
scriptdata['name'] = scriptdata[scriptdata.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)


In [41]:
scriptdata['name']=scriptdata.name.str.replace('\d+', '')


In [42]:
scriptdata['name']= scriptdata['name'].str.replace('[!”#$%&’()*+,-./:;<=>?@[\]^_`{}~]'.format(string.punctuation), " ")


In [43]:
scriptdata= scriptdata['name'].str.split()
script= scriptdata.values.tolist()


In [44]:
from functools import reduce
flat_list = reduce(lambda x, y: x+y, script)


In [45]:
script = list(dict.fromkeys(flat_list))


In [46]:
str1 = ' '.join(script)


In [47]:
nlp = spacy.load("en")
doc = nlp(str1) 
mylist = []
for token in doc: 
      mylist.append((token, token.pos_))


In [48]:
m1_inc = 0
f1_inc = 0
o1_inc = 0
wor_out1 = []
wor_out2 =  []
wor_out3 = []
for word in mylist:
    if word[-1] in ['INTJ','ADJ','ADV','DET','PRON']:
        wor_out2.append(word)
        f1_inc = f1_inc + 1
  
    elif word[-1] in ['VERBS','NOUN','PROPN','ADP','AUXILARY','CCONJ','SCONJ','PART','prep']:
        wor_out1.append(word)
        m1_inc = m1_inc + 1
        
    elif word[-1] in ['SYM','PUNCT','NUM','X']:
        wor_out3.append(word)
        o1_inc = o1_inc + 1
    
        
        
                
                



In [52]:
from operator import itemgetter
res1 = list(map(itemgetter(0), wor_out1)) 
res2 = list(map(itemgetter(0), wor_out2)) 


In [53]:
wordmkey=[str(x) for x in res1]
wordfkey=[str(y) for y in res2]



In [54]:
unwanted_num = {'a', 'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','y','z'} 
kmk = [ele for ele in wordmkey if ele not in unwanted_num] 
kfk = [ele for ele in wordfkey if ele not in unwanted_num] 


In [55]:
v1 = 0
c1 = 0
v2 = 0
c2 = 0
c_f1 = []
v_f1 =  []
c_m2 = []
v_m2 =  []
for word in wordfkey:
    if word[-1] in ['a','e','i','o','u']:
        v_f1.append(word)
        v1 = v1 + 1
  
    else:
        c_f1.append(word)
        c1 = c1 + 1
        
for word in wordmkey:
    if word[-1] in ['a','e','i','o','u']:
        v_m2.append(word)
        v2 = v2 + 1
  
    else:
        c_m2.append(word)
        c2 = c2 + 1
        
                
                
                


In [56]:
word_cat1  =   c_m2 + c_f1
word_cat2   =   v_m2 + v_f1


In [57]:
import pandas as pd
type1 = pd.DataFrame(word_cat2)
type2 = pd.DataFrame(word_cat1)
type1.rename(columns = {0:'name'}, inplace = True) 
type2.rename(columns = {0:'name'}, inplace = True) 


In [58]:
type1['gender'] = 'F'
type2['gender'] = 'M'
original = type2.append(type1) 
original['name'] = original['name'].str.capitalize() 


In [59]:
original.head(2)


,name,gender
0,Facultyplus,M
1,Faculty,M


In [60]:
df_names = original
df_names.gender.replace({'F':0,'M':1},inplace=True)


In [61]:
Xfeatures =df_names['name']
cv = CountVectorizer()
X = cv.fit_transform(Xfeatures)
gender_vectorizer = open("gender_vectorizer.pkl","wb")
joblib.dump(cv,gender_vectorizer)
gender_vectorizer.close()
cv.get_feature_names()


['aaa',
 'abdul',
 'about',
 'abs',
 'academic',
 'academy',
 'accountant',
 'across',
 'adamas',
 'adani',
 'address',
 'adi',
 'adikavi',
 'aditya',
 'admin',
 'admission',
 'advertisement',
 'aeronautical',
 'agartala',
 'agra',
 'agricultural',
 'agriculture',
 'ahmed',
 'ahmedabad',
 'ajk',
 'ajkim',
 'ajmer',
 'akal',
 'alagar',
 'alagesan',
 'aligarh',
 'allahabad',
 'aloysius',
 'alwin',
 'am',
 'ambal',
 'ambassador',
 'ambedkar',
 'ambiga',
 'amity',
 'ammal',
 'amman',
 'amravati',
 'amrita',
 'amritsar',
 'an',
 'anand',
 'ananthapuramu',
 'anchor',
 'and',
 'andal',
 'andhra',
 'angappa',
 'animal',
 'anna',
 'annamacharya',
 'annamalai',
 'announcement',
 'anurag',
 'anusandhan',
 'apeejay',
 'apollo',
 'application',
 'applications',
 'aps',
 'aravindar',
 'architecture',
 'arjun',
 'art',
 'article',
 'arts',
 'arulmurugan',
 'aruppukottai',
 'aryan',
 'as',
 'asansol',
 'assam',
 'assistant',
 'associate',
 'associated',
 'association',
 'at',
 'atria',
 'august',
 'au

In [62]:
from sklearn.naive_bayes import MultinomialNB
X
y = df_names.name
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = MultinomialNB()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
print("Accuracy of Model",clf.score(X_test,y_test)*100,"%")
print("Accuracy of Model",clf.score(X_train,y_train)*100,"%")


Accuracy of Model 0.0 %
Accuracy of Model 0.7259528130671506 %


In [63]:
def features(name):
    name = name.lower()
    return {
        'first-letter': name[0], 
        'first2-letters': name[0:2], 
        'first3-letters': name[0:3], 
        'last-letter': name[-1],
        'last2-letters': name[-2:],
        'last3-letters': name[-3:],
    }
features = np.vectorize(features)
df_X = features(df_names['name'])
df_y = df_names['gender']
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(df_X, df_y, test_size=0.50, random_state=42)
dv = DictVectorizer()
dv.fit_transform(dfX_train)
dclf = DecisionTreeClassifier()
my_xfeatures =dv.transform(dfX_train)
dclf.fit(my_xfeatures, dfy_train)
print(dclf.score(dv.transform(dfX_train), dfy_train)) 
print(dclf.score(dv.transform(dfX_test), dfy_test))

1.0
1.0


In [65]:
DecisionTreeModel = open("DecisionTreeModel.pkl","wb")
joblib.dump(dclf,DecisionTreeModel)
DecisionTreeModel.close()
NaiveBayesModel = open("NaiveBayesModel.pkl","wb")
joblib.dump(clf,NaiveBayesModel)
NaiveBayesModel.close()